<a href="https://colab.research.google.com/github/naku2/smartFactory/blob/main/MVtec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
from tensorflow import keras
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Input, Conv2D, Dense, Flatten, MaxPooling2D
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from PIL import Image

### 데이터 불러오기

In [ ]:
input_folder = '/content/drive/MyDrive/transistor/train/good'
output_folder = '/content/drive/MyDrive/transistor/train/good_scaled'

image_files = os.listdir(input_folder)

for image_file in image_files:
  image_path = os.path.join(input_folder, image_file)

  image = Image.open(image_path)

  resized_image = image.resize((640, 640))

  image_array = np.array(resized_image)

  scaled_image_array = image_array / 255.0

  output_path = os.path.join(output_folder, image_file)

  np.save(output_path, scaled_image_array)

  image.close()

In [ ]:
output_files = os.listdir(output_folder)

In [ ]:
scaled_image_array.dtype

dtype('float64')

이미지 전처리 후 라벨링

In [ ]:
for output_file in output_files:
  print(output_file)

010.png.npy
014.png.npy
020.png.npy
031.png.npy
028.png.npy
033.png.npy
034.png.npy
006.png.npy
017.png.npy
002.png.npy
023.png.npy
004.png.npy
008.png.npy
032.png.npy
005.png.npy
035.png.npy
025.png.npy
019.png.npy
036.png.npy
021.png.npy
029.png.npy
011.png.npy
000.png.npy
003.png.npy
001.png.npy
016.png.npy
038.png.npy
039.png.npy
026.png.npy
015.png.npy
037.png.npy
022.png.npy
009.png.npy
024.png.npy
027.png.npy
018.png.npy
013.png.npy
030.png.npy
055.png.npy
012.png.npy
007.png.npy
060.png.npy
075.png.npy
099.png.npy
071.png.npy
083.png.npy
079.png.npy
062.png.npy
111.png.npy
051.png.npy
053.png.npy
105.png.npy
089.png.npy
047.png.npy
091.png.npy
101.png.npy
104.png.npy
057.png.npy
063.png.npy
068.png.npy
056.png.npy
084.png.npy
113.png.npy
058.png.npy
078.png.npy
082.png.npy
043.png.npy
102.png.npy
112.png.npy
065.png.npy
085.png.npy
052.png.npy
054.png.npy
059.png.npy
103.png.npy
086.png.npy
046.png.npy
110.png.npy
092.png.npy
044.png.npy
088.png.npy
050.png.npy
087.png.npy
081.

In [ ]:
import csv

label_file = "label_files.csv"

fieldnames = ["filename", "label"]

label_data = []

for output_file in output_files:
  label_data.append({"filename": output_file, "label": 0}) #0 = good

with open(label_file, 'w', newline='') as f:
  writer = csv.DictWriter(f, fieldnames=fieldnames)
  writer.writeheader()
  writer.writerows(label_data)

In [ ]:
#무조건 csv로 만들어야 편하고 좋구나

In [ ]:
print(label_data)

[{'filename': '010.png.npy', 'label': 0}, {'filename': '014.png.npy', 'label': 0}, {'filename': '020.png.npy', 'label': 0}, {'filename': '031.png.npy', 'label': 0}, {'filename': '028.png.npy', 'label': 0}, {'filename': '033.png.npy', 'label': 0}, {'filename': '034.png.npy', 'label': 0}, {'filename': '006.png.npy', 'label': 0}, {'filename': '017.png.npy', 'label': 0}, {'filename': '002.png.npy', 'label': 0}, {'filename': '023.png.npy', 'label': 0}, {'filename': '004.png.npy', 'label': 0}, {'filename': '008.png.npy', 'label': 0}, {'filename': '032.png.npy', 'label': 0}, {'filename': '005.png.npy', 'label': 0}, {'filename': '035.png.npy', 'label': 0}, {'filename': '025.png.npy', 'label': 0}, {'filename': '019.png.npy', 'label': 0}, {'filename': '036.png.npy', 'label': 0}, {'filename': '021.png.npy', 'label': 0}, {'filename': '029.png.npy', 'label': 0}, {'filename': '011.png.npy', 'label': 0}, {'filename': '000.png.npy', 'label': 0}, {'filename': '003.png.npy', 'label': 0}, {'filename': '0

bad image 파일명 변경

In [ ]:
folder_path = '/content/drive/MyDrive/transistor/test/bent_lead'
extension = '.png'
prefix = 'bent_lead_'

for i, filename in enumerate(os.listdir(folder_path)):
  new_filename = f'{prefix}{i+1:03d}{extension}'

  old_path = os.path.join(folder_path, filename)
  new_path = os.path.join(folder_path, new_filename)

  os.rename(old_path, new_path)


folder_path = '/content/drive/MyDrive/transistor/test/cut_lead'
extension = '.png'
prefix = 'cut_lead_'

for i, filename in enumerate(os.listdir(folder_path)):
  new_filename = f'{prefix}{i+1:03d}{extension}'

  old_path = os.path.join(folder_path, filename)
  new_path = os.path.join(folder_path, new_filename)

  os.rename(old_path, new_path)


folder_path = '/content/drive/MyDrive/transistor/test/damaged_case'
extension = '.png'
prefix = 'damaged_'

for i, filename in enumerate(os.listdir(folder_path)):
  new_filename = f'{prefix}{i+1:03d}{extension}'

  old_path = os.path.join(folder_path, filename)
  new_path = os.path.join(folder_path, new_filename)

  os.rename(old_path, new_path)


folder_path = '/content/drive/MyDrive/transistor/test/misplaced'
extension = '.png'
prefix = 'misplaced_'

for i, filename in enumerate(os.listdir(folder_path)):
  new_filename = f'{prefix}{i+1:03d}{extension}'

  old_path = os.path.join(folder_path, filename)
  new_path = os.path.join(folder_path, new_filename)

  os.rename(old_path, new_path)

###bad image 전처리 후 라벨링

In [ ]:
input_folder = '/content/drive/MyDrive/transistor/test/bent_lead'
output_folder = '/content/drive/MyDrive/transistor/train/bent_lead_scaled'

image_files = os.listdir(input_folder)

for image_file in image_files:
  image_path = os.path.join(input_folder, image_file)

  image = Image.open(image_path)

  resized_image = image.resize((640, 640))

  image_array = np.array(resized_image)

  scaled_image_array = image_array / 255.0

  output_path = os.path.join(output_folder, image_file)

  np.save(output_path, scaled_image_array)

  image.close()

In [ ]:
input_folder = '/content/drive/MyDrive/transistor/test/cut_lead'
output_folder = '/content/drive/MyDrive/transistor/train/cut_lead_scaled'

image_files = os.listdir(input_folder)

for image_file in image_files:
  image_path = os.path.join(input_folder, image_file)

  image = Image.open(image_path)

  resized_image = image.resize((640, 640))

  image_array = np.array(resized_image)

  scaled_image_array = image_array / 255.0

  output_path = os.path.join(output_folder, image_file)

  np.save(output_path, scaled_image_array)

  image.close()

In [ ]:
input_folder = '/content/drive/MyDrive/transistor/test/damaged_case'
output_folder = '/content/drive/MyDrive/transistor/train/damaged_case_scaled'

image_files = os.listdir(input_folder)

for image_file in image_files:
  image_path = os.path.join(input_folder, image_file)

  image = Image.open(image_path)

  resized_image = image.resize((640, 640))

  image_array = np.array(resized_image)

  scaled_image_array = image_array / 255.0

  output_path = os.path.join(output_folder, image_file)

  np.save(output_path, scaled_image_array)

  image.close()

In [ ]:
input_folder = '/content/drive/MyDrive/transistor/test/misplaced'
output_folder = '/content/drive/MyDrive/transistor/train/misplaced_scaled'

image_files = os.listdir(input_folder)

for image_file in image_files:
  image_path = os.path.join(input_folder, image_file)

  image = Image.open(image_path)

  resized_image = image.resize((640, 640))

  image_array = np.array(resized_image)

  scaled_image_array = image_array / 255.0

  output_path = os.path.join(output_folder, image_file)

  np.save(output_path, scaled_image_array)

  image.close()

bad image => bad_scaled로 취합

In [ ]:
import shutil

source_folder = '/content/drive/MyDrive/transistor/train/bent_lead_scaled'
destination_folder = '/content/drive/MyDrive/transistor/train/bad_scaled'

for filename in os.listdir(source_folder):
  source_path = os.path.join(source_folder, filename)
  destination_path = os.path.join(destination_folder, filename)
  shutil.copy(source_path, destination_path)

In [ ]:
source_folder = '/content/drive/MyDrive/transistor/train/cut_lead_scaled'

for filename in os.listdir(source_folder):
  source_path = os.path.join(source_folder, filename)
  destination_path = os.path.join(destination_folder, filename)
  shutil.copy(source_path, destination_path)

source_folder = '/content/drive/MyDrive/transistor/train/damaged_case_scaled'

for filename in os.listdir(source_folder):
  source_path = os.path.join(source_folder, filename)
  destination_path = os.path.join(destination_folder, filename)
  shutil.copy(source_path, destination_path)

source_folder = '/content/drive/MyDrive/transistor/train/misplaced_scaled'

for filename in os.listdir(source_folder):
  source_path = os.path.join(source_folder, filename)
  destination_path = os.path.join(destination_folder, filename)
  shutil.copy(source_path, destination_path)

bad image labelling

In [ ]:
label_file

'label_files.csv'

In [ ]:
label_data

[{'filename': '010.png.npy', 'label': 0},
 {'filename': '014.png.npy', 'label': 0},
 {'filename': '020.png.npy', 'label': 0},
 {'filename': '031.png.npy', 'label': 0},
 {'filename': '028.png.npy', 'label': 0},
 {'filename': '033.png.npy', 'label': 0},
 {'filename': '034.png.npy', 'label': 0},
 {'filename': '006.png.npy', 'label': 0},
 {'filename': '017.png.npy', 'label': 0},
 {'filename': '002.png.npy', 'label': 0},
 {'filename': '023.png.npy', 'label': 0},
 {'filename': '004.png.npy', 'label': 0},
 {'filename': '008.png.npy', 'label': 0},
 {'filename': '032.png.npy', 'label': 0},
 {'filename': '005.png.npy', 'label': 0},
 {'filename': '035.png.npy', 'label': 0},
 {'filename': '025.png.npy', 'label': 0},
 {'filename': '019.png.npy', 'label': 0},
 {'filename': '036.png.npy', 'label': 0},
 {'filename': '021.png.npy', 'label': 0},
 {'filename': '029.png.npy', 'label': 0},
 {'filename': '011.png.npy', 'label': 0},
 {'filename': '000.png.npy', 'label': 0},
 {'filename': '003.png.npy', 'labe

In [ ]:
output_folder = '/content/drive/MyDrive/transistor/train/bad_scaled'

output_files = os.listdir(output_folder)


for output_file in output_files:
  label_data.append({"filename": output_file, "label": 1})

  with open(label_file, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(label_data)

### 모델 생성

In [ ]:
train = pd.read_csv('/content/label_files.csv')

In [ ]:
train.shape

(253, 2)

In [ ]:
train['filename'].shape

(253,)

In [ ]:
#csv파일에는 이미지 관련 데이터가 따로 들어가지 않는다. 따라서 파일명에 따른 경로를 따로 지정해줘야함

라펠링 파일 취합

In [ ]:
source_folder = '/content/drive/MyDrive/transistor/train/good_scaled'  # 복사할 파일들이 있는 폴더 경로
destination_folder = '/content/drive/MyDrive/transistor/train/test_scaled'  # 복사한 파일들을 저장할 폴더 경로

# 대상 폴더가 없으면 생성
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# 폴더 내의 파일들을 복사
for filename in os.listdir(source_folder):
    source_path = os.path.join(source_folder, filename)
    destination_path = os.path.join(destination_folder, filename)
    shutil.copy(source_path, destination_path)

source_folder = '/content/drive/MyDrive/transistor/train/bad_scaled'  # 복사할 파일들이 있는 폴더 경로

# 대상 폴더가 없으면 생성
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# 폴더 내의 파일들을 복사
for filename in os.listdir(source_folder):
    source_path = os.path.join(source_folder, filename)
    destination_path = os.path.join(destination_folder, filename)
    shutil.copy(source_path, destination_path)

라벨링파일 불러오기

In [ ]:
from keras.utils import to_categorical

label_file = pd.read_csv('/content/label_files.csv')

x = []
y = []
for i in range(len(label_file)):
  file_path = label_file['filename'][i]
  # print(type(file_path))
  label = label_file['label'][i]

  loaded_data = np.load('/content/drive/MyDrive/transistor/train/test_scaled/' + file_path)
  x.append(loaded_data)
  y.append(label)

x = np.array(x)
y = np.array(y)

y = to_categorical(y)

In [ ]:
print(x)

In [ ]:
num_classes = 2

# Sequential 모델 구성
model = tf.keras.Sequential()

# 입력 레이어
model.add(Input(shape=(640, 640, 3)))

# 컨볼루션 레이어
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# 플래튼(Flatten) 레이어
model.add(Flatten())

# 완전 연결(Dense) 레이어
model.add(Dense(64, activation='relu'))

# 출력 레이어
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 요약 정보 출력
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 638, 638, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 319, 319, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 317, 317, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 158, 158, 64)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 156, 156, 128)     73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 78, 78, 128)     

모델 훈련

In [ ]:
history = model.fit(x, y, validation_split=0.2, epochs=100, verbose=1, batch_size=8)

In [ ]:
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

NameError: ignored

데이터 수 증강

In [ ]:
rotated_data = np.rot90(x)

np.save('rotated_data.npy', rotated_data)
np.save('rotated_data_label.npy', y)
#(640, 253, 640, 3)

flipped_data_lr = np.flip(x, axis=1)

np.save('flipped_data_lr.npy', flipped_data_lr)
np.save('flipped_data_lr_label.npy', y)

flipped_data_ud = np.flip(x, axis=0)

np.save('flipped_data_ud.npy', flipped_data_ud)
np.save('flipped_data_ud_label.npy', y)

mean = 0
stddev = 0.05
noise = np.random.normal(mean, stddev, x.shape)
noisy_data = x + noise
np.save('noisy_data.npy', noisy_data)
np.save('noisy_data_label.npy', y)

취합 후 시뮬

In [ ]:
flipped_data_lr.head()

AttributeError: ignored

In [ ]:
flipped_data_lr = np.load('flipped_data_lr.npy')
flipped_data_ud = np.load('flipped_data_ud.npy')
noisy_data = np.load('noisy_data.npy')
flipped_data_lr_label = np.load('flipped_data_lr_label.npy')
flipped_data_ud_label = np.load('flipped_data_ud_label.npy')
noisy_data_label = np.load('noisy_data_label.npy')

total_x = np.concatenate((x, flipped_data_lr, flipped_data_ud, noisy_data), axis=0)
total_y = np.concatenate((y, flipped_data_lr_label, flipped_data_ud_label, noisy_data_label), axis=0)

